# CJMLS

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from IPython.core.display import display, HTML
from concurrent.futures import ThreadPoolExecutor

pd.set_option('display.max_rows', None)

def preview_cjmls_df(mydf, save=False):
    columns = ['address', 'city', 'county', 'bedrooms', 'baths_full', 'baths_part',
               'price', 'sqft',
               'park_and_ride_name',
               'park_and_ride_duration_text', 'park_and_ride_type', 'park_and_ride_distance',
               'park_and_ride_bus_name', 'park_and_ride_bus_distance',
               'barnabas_duration_text', 'lat', 'lng',]
    if save:
        mydf[columns].to_csv('cjmls.csv')
    display(HTML(
        """<style>.dataframe td {white-space: nowrap;}</style>""" +
        mydf[columns].to_html(
            formatters={
                '__index__':
                    lambda id: f'<a target="_blank" href="https://cjmls.herokuapp.com/properties/{id}">{id}</a>'},
            escape=False)))

**Import global settings**

In [3]:
from settings import (blacklist_cities, MIN_PRICE, MAX_PRICE, MAX_TAX, MIN_BEDS, MIN_BATHS,
                      ONE_HOUR, HALF_HOUR, TEN_MINUTES,)

print(MIN_PRICE, MAX_PRICE, MAX_TAX, MIN_BEDS, MIN_BATHS,)

300000 525000 16000 3 2


**Download listings from [CJMLS](https://www.mcmls.net/)**

In [4]:
from cjmls import get_listings

listings = [listing for meta, listing in get_listings(min_beds=MIN_BEDS, min_baths=MIN_BATHS, max_price=MAX_PRICE)]
print(f"Downloaded {len(listings)} listings from CJMLS.")

Downloaded 1023 listings from CJMLS.


**Geocode each address using Google Maps API**

In [5]:
from geolocate import add_geocode_to_listing
for listing in tqdm(listings):
    if not listing.get('address'): continue
    add_geocode_to_listing(listing)

HBox(children=(IntProgress(value=0, max=1023), HTML(value='')))

could not geocode: 0 Samarpan Court Robbinsville, NJ or 0 Samarpan Court Robbinsville, NJ
could not geocode: 0 Samarpan Court Robbinsville, NJ or 0 Samarpan Court Robbinsville, NJ
could not geocode: 118 Swarthmore Terrace Menlo Park Terrace, NJ or 118 Swarthmore Terrace Menlo Park Terrace, NJ
could not geocode: 43 Traditions Place Monroe, NJ or 43 Traditions Place Monroe, NJ
could not geocode: 8 Traditions Place Monroe, NJ or 8 Traditions Place Monroe, NJ
could not geocode: 795 Chalet Drive Woodbridge Proper, NJ or 795 Chalet Drive Woodbridge Proper, NJ
could not geocode: 8 Ridgedale Place Woodbridge Proper, NJ or 8 Ridgedale Place Woodbridge Proper, NJ
could not geocode: 155 S Park Drive Woodbridge Proper, NJ or 155 S Park Drive Woodbridge Proper, NJ
could not geocode: 54 Mercer Street Menlo Park Terrace, NJ or 54 Mercer Street Menlo Park Terrace, NJ
could not geocode: 15 Colonial Court Woodbridge Proper, NJ or 15 Colonial Court Woodbridge Proper, NJ
could not geocode: 169 Hillside Av

**Find closest NJ Transit Park and Ride to each address**

In [6]:
from commute import add_closest_park_and_ride_to_listing

for i in tqdm(range(len(listings))):
    listing = listings[i]
    add_closest_park_and_ride_to_listing(listing)

HBox(children=(IntProgress(value=0, max=1023), HTML(value='')))

**Get commute time for each address through both park and ride and walking**

In [7]:
from commute import add_commute_to_listing

for i in tqdm(range(len(listings))):
    listing = listings[i]
    add_commute_to_listing(listing)

print(sum(1 for listing in listings if listing.get('park_and_ride_duration')),
      "out of", len(listings), "have park and ride commute times.")

print(sum(1 for listing in listings if listing.get('nyc_duration')),
  "out of", len(listings), "have walking commute times.")

HBox(children=(IntProgress(value=0, max=1023), HTML(value='')))


990 out of 1023 have park and ride commute times.
906 out of 1023 have walking commute times.


In [8]:
df = pd.DataFrame(listings)
df.set_index('id', inplace=True)
df = df[
    (~df.city.isin(blacklist_cities)) &
    (df.barnabas_duration < HALF_HOUR + TEN_MINUTES) &
    (df.park_and_ride_duration1 < TEN_MINUTES) &
    (df.park_and_ride_duration < ONE_HOUR + TEN_MINUTES) &
    (df.price > MIN_PRICE)
]
df = df.sort_values(by=['price',], ascending=[False])
print(len(df), "listings after filtering")
preview_cjmls_df(df)

109 listings after filtering


,address,city,county,bedrooms,baths_full,baths_part,price,sqft,park_and_ride_name,park_and_ride_duration_text,park_and_ride_type,park_and_ride_distance,park_and_ride_bus_name,park_and_ride_bus_distance,barnabas_duration_text,lat,lng
id,,,,,,,,,,,,,,,,,
1800267,11 Regina Street,Woodbridge Township,Middlesex,4,2,1,525000,NaN,Woodbridge,"6 mins, 43 mins",rail,1.052740,Carteret,3.894006,37 mins,40.5651,-74.2943
1804757,166 Sherwood Road,Woodbridge Township,Middlesex,3,2,1,525000,NaN,Metropark,"7 mins, 38 mins",rail,1.776928,Carteret,4.613488,36 mins,40.5923,-74.3161
1806028,79 Demorest Avenue,Woodbridge Township,Middlesex,4,2,1,509000,NaN,Avenel,"4 mins, 36 mins",rail,0.725264,Carteret,3.444604,36 mins,40.5768,-74.2913
1805257,201 Maple Avenue,Metuchen,Middlesex,5,2,2,500000,NaN,Metuchen,"3 mins, 51 mins",rail,0.466037,Sayreville,5.861296,38 mins,40.5475,-74.3616
1808973,50 Terry Avenue,Edison,Middlesex,3,2,1,499900,NaN,Metropark,"4 mins, 38 mins",rail,1.046095,Sayreville,6.196405,35 mins,40.5598,-74.3456
1802582,47 Normandy Drive,Woodbridge Township,Middlesex,5,3,1,499900,3705.0,Metropark,"10 mins, 38 mins",rail,2.482587,Watchung Park & Ride,4.909467,36 mins,40.6045,-74.3331
1808336,21 Commander Patrick Dunn Court,Woodbridge Township,Middlesex,4,4,0,499900,NaN,Woodbridge,"8 mins, 43 mins",rail,2.149547,Sayreville,4.259026,37 mins,40.5386,-74.311
1808833,223 New Dover Road,Woodbridge Township,Middlesex,4,3,0,499000,NaN,Avenel,"5 mins, 36 mins",rail,1.346549,Carteret,3.902292,34 mins,40.5834,-74.302
1810586,107 Cooper Avenue,Woodbridge Township,Middlesex,4,3,0,499000,1930.0,Metropark,"4 mins, 38 mins",rail,0.472839,Carteret,5.056894,33 mins,40.5699,-74.3206


In [10]:
import folium

home = (40.8961863, -74.1726829,)

m = folium.Map(location=home, zoom_start=10)

folium.Marker(home, tooltip=f'<i>115 Dumont Ave. Clifton, NJ</i>', icon=folium.Icon(color='green')).add_to(m)

for id, row in df.iterrows():
    url = f"https://cjmls.herokuapp.com/properties/{id}"
    address = row.address + ", " + row.city
    popup = f"""
    Address: {address}<br/>
    MLS: <a target="_blank" href="{url}">{id}</a><br/>
    Price: {row.price}<br/>
    Dad (Park and Ride): {row.park_and_ride_duration_text}<br/>
    Dad (Walk and Ride): {row.nyc_duration_text}<br/>
    Mom: {row.barnabas_duration_text}
    """
    folium.Marker((row.lat, row.lng,), popup=f'<i>{popup}</i>', tooltip=f'<i>{address}</i>').add_to(m)

m